In [13]:
import yfinance as yf
import numpy as np
import pandas as pd
tsla = yf.download('TSLA', start='2019-01-01', end='2025-03-05')
xly = yf.download('XLY', start='2019-01-01', end='2025-03-05')
spy = yf.download('SPY', start='2019-01-01', end='2025-03-05')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [14]:
import requests
import pandas as pd

url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&date_from=20250101T0130&date_to=20250301T0130&limit=1000&tickers=TSLA&apikey=PNM5EHRALIOT1CKJ'

response = requests.get(url)

if response.status_code == 200:
    sentiment_data = response.json()
    sentiment_df = pd.DataFrame(sentiment_data['feed']) 
    print(sentiment_df.head())
else:
    print("API call failed:", response.status_code)

sentiment_json = response.json()
print(sentiment_json)

                                               title  \
0  Elon Musk must face Twitter shareholders' laws...   
1  Elon Musk Folds X Into xAI, Creating a $113 Bi...   
2  Trump's 25% Auto Tariffs Rattle Wall Street, F...   
3                       How to Tune Out Market Noise   
4  EXCLUSIVE: Which Magnificent 7 Stock Will Perf...   

                                                 url   time_published  \
0  https://www.cnbc.com/2025/03/28/elon-musk-must...  20250329T002038   
1  https://decrypt.co/312221/elon-musk-x-xai-113-...  20250328T232829   
2  https://www.benzinga.com/news/25/03/44546899/t...  20250328T205510   
3  https://www.fool.com/investing/2025/03/28/how-...  20250328T194400   
4  https://www.benzinga.com/tech/25/03/44544129/e...  20250328T190544   

               authors                                            summary  \
0                   []  A proposed class-action lawsuit against Elon M...   
1       [Jason Nelson]  The merger between Elon Musk's X and xAI creat

In [24]:
sentiment_data = []
for news_item in sentiment_json.get("feed", []):
    sentiment_data.append({
            "time_published": pd.to_datetime(news_item["time_published"]),
            "sentiment_score": news_item["overall_sentiment_score"],
            "sentiment_label": news_item["overall_sentiment_label"],
    })
sentiment_data = pd.DataFrame(sentiment_data)

In [25]:
import json

# Save raw API response
with open("TSLA_sentiment_raw.json", "w") as f:
    json.dump(sentiment_json, f, indent=4)

print("Full sentiment JSON saved ✅")


Full sentiment JSON saved ✅


In [16]:
sentiment_data = []
for news_item in sentiment_json.get("feed", []):
    sentiment_data.append({
            "time_published": pd.to_datetime(news_item["time_published"]),
            "sentiment_score": news_item["overall_sentiment_score"],
            "sentiment_label": news_item["overall_sentiment_label"],
    })
sentiment_data = pd.DataFrame(sentiment_data)

In [17]:
sentiment_data['time_published'] = sentiment_data['time_published'].dt.date

In [18]:
pd.to_datetime(sentiment_data['time_published'])

0     2025-03-29
1     2025-03-28
2     2025-03-28
3     2025-03-28
4     2025-03-28
         ...    
670   2025-03-05
671   2025-03-05
672   2025-03-05
673   2025-03-05
674   2025-03-05
Name: time_published, Length: 675, dtype: datetime64[ns]

In [19]:
sentiment_scores_filtered = sentiment_data[pd.to_datetime(sentiment_data['time_published']).isin(tsla.index)]

In [20]:
# Fix the multi-level column issue by selecting the 'Volume' column and resetting its name
tsla_volume = tsla[('Volume', 'TSLA')].rename('Volume')

# Ensure the index of tsla_volume is a column and convert it to match the type of time_published
tsla_volume = tsla_volume.reset_index()
tsla_volume['Date'] = pd.to_datetime(tsla_volume['Date'])

# Convert time_published to datetime
sentiment_scores_filtered['time_published'] = pd.to_datetime(sentiment_scores_filtered['time_published'])

# Merge the dataframes
merged_data = pd.merge(tsla_volume, sentiment_scores_filtered, left_on='Date', right_on='time_published', how='inner')

In [21]:
merged_data

,Date,Volume,time_published,sentiment_score,sentiment_label


In [22]:
merged_data['Weighted_Sentiment'] = merged_data['Volume'] * merged_data['sentiment_score']
merged_data['5_day_avg_sentiment'] = merged_data['Weighted_Sentiment'].rolling(window=5).mean()
merged_data['Buy_Condition'] = merged_data['5_day_avg_sentiment'] > 0

In [23]:
print(merged_data[['Volume', 'sentiment_score', '5_day_avg_sentiment', 'Buy_Condition']])

Empty DataFrame
Columns: [Volume, sentiment_score, 5_day_avg_sentiment, Buy_Condition]
Index: []
